In [ ]:
# Desired Imports
import torch
import tqdm
from tqdm import trange
from transformers import (AdamW, GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, get_linear_schedule_with_warmup)
from torch.utils.data import Dataset
import pickle
import numpy as np
from collections import defaultdict
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import os
import shutil
import subprocess
import json
import torch.nn.utils as F
from transformers import WEIGHTS_NAME
import glob
import pandas as pd
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Directories needed
qna_dataset_dir = "/content/drive/MyDrive/QnA"
qna_model_chkpts_dir =  "/content/drive/MyDrive/QnA/"
final_qna_model_dir = "/content/drive/MyDrive/QnA/final_QnA"

In [ ]:
# Required Arguments
args_dir = {
  "save_steps" : 20, # can be changed
  "num_epochs" : 3,
  "gradient_accumulation_steps" : 10,
  "adam_epsilon" : 1e-8,
  "warmup_steps" : 0,
  "learning_rt" : 5e-5,
  "max_grad_norm" : 1.0,
  "data_dir" : qna_dataset_dir,
  "model_type" : "gpt2",
  "model_name" : "gpt2",  # set to gtp2-large
  "train_batch_size" : 5,
  "eval_batch_size" : 5,
  "extra_embedding_dim" : 768,
  "global_dense_feature_list" : None # in file it will be saved with the value null; while reading take care of this thing
}

model_type = args_dir["model_type"]
model_name = args_dir["model_name"]
data_dir = args_dir["data_dir"]
save_steps = args_dir["save_steps"]
num_epochs = args_dir["num_epochs"]
gradient_accumulation_steps = args_dir["gradient_accumulation_steps"]
adam_epsilon = args_dir["adam_epsilon"]
warmup_steps = args_dir["warmup_steps"]
train_batch_size = args_dir["train_batch_size"]
eval_batch_size = args_dir["eval_batch_size"]
learning_rt = args_dir["learning_rt"]
extra_embedding_dim = args_dir["extra_embedding_dim"] # Size of linear layer used for projecting extra embeddings.
global_dense_feature_list = args_dir["global_dense_feature_list"]
max_grad_norm = args_dir["max_grad_norm"]

In [ ]:
# Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()

print("Device- ", device)
print("No. of GPUs- ", n_gpus)

Device-  cuda
No. of GPUs-  1


In [ ]:
# Install Transformers
!pip install transformers

In [ ]:
# Initialize model classes variables
MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]

print("GPT2 Config class- ", config_class)
print("GPT2 Model class- ", model_class)
print("GPT2 Tokenizer class- ", tokenizer_class)

GPT2 Config class-  <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>
GPT2 Model class-  <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
GPT2 Tokenizer class-  <class 'transformers.models.gpt2.tokenization_gpt2.GPT2Tokenizer'>


In [ ]:
# Init config
config = config_class.from_pretrained(model_name)
print("GPT2Config loaded")

# Init model
model = model_class.from_pretrained(model_name, config = config)
print("GPT2LMHeadModel loaded")

# Init tokenizer
tokenizer = tokenizer_class.from_pretrained(model_name,do_lower_case = False)
print("GPT2Tokenizer loaded")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

GPT2Config loaded


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel loaded


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2Tokenizer loaded


In [ ]:
# adding extra_dimension to config --- No need in DVP ---- Can be removed later
config.extra_embedding_dim = extra_embedding_dim # don't know why we are using it; -- explore it

In [ ]:
# Add special tokens to tokenizer
SPECIAL_TOKENS = {
    "additional_special_tokens": ["<segment_1>", "<segment_2>"],
    "pad_token": "<pad>",
    "bos_token": "<bos>",
    "eos_token": "<eos>"
}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
print("Special Tokens addded to tokenizer")

print("Total tokens- ", len(tokenizer))

Special Tokens addded to tokenizer
Total tokens-  50262


In [ ]:
# resize token embedding matrix to take care of special tokens added
model.resize_token_embeddings(len(tokenizer)) # each token of size-> 1280(gpt2-large), 768(gpt2)

Embedding(50262, 768)

In [ ]:
# move model to device
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50262, bias=False)
)

In [ ]:
# Init configs

MAX_PARAPHRASE_LEN = 100

# mainly to handle input
INPUT_FORMAT_CONFIG = {
    "keys": [
        {"key": "sent1_tokens", "position": 0},
        {"key": "sent2_tokens", "position": 1}
    ],
    "max_prefix_length": int(MAX_PARAPHRASE_LEN / 2),
    "max_suffix_length": int(MAX_PARAPHRASE_LEN / 2)
}

In [ ]:
# Fn to convert example input to dictionary
def input_to_dict(config, sample, tokenizer):
    example = {}

    for inp_key in config["keys"]:
        val = sample[inp_key["position"]]
        example[inp_key["key"]] = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(val))

    return example

In [ ]:
# Preprocess input from paranmt
def preprocess(exp, tokenizer, config, do_tokenize=True):
  MASK_TOKEN_ID = -100

  max_prefix_len = config["max_prefix_length"]
  max_suffix_len = config["max_suffix_length"]

  if do_tokenize:
    sent1 = np.array(exp["sent1_tokens"])
    sent2 = np.array(exp["sent2_tokens"])

  # truncate
  if(len(sent1) > max_prefix_len):
    sent1 = sent1[:max_prefix_len]

  if(len(sent2) > max_suffix_len):
    sent2 = sent2[:max_suffix_len]

  # add padding; left padding to prefix and right padding to suffix
  count_pad_tokens_prefix = max_prefix_len - len(sent1)
  sent1 = np.pad(sent1, (count_pad_tokens_prefix, 0), constant_values = tokenizer.pad_token_id)

  # add <eos> to suffix
  sent2 = np.append(sent2, tokenizer.eos_token_id)

  count_pad_tokens_suffix = (max_suffix_len + 1) - len(sent2)
  sent2 = np.pad(sent2, (0, count_pad_tokens_suffix), constant_values = tokenizer.pad_token_id)

  # sentence to input gpt2
  sentence_to_input_gpt2 = np.concatenate([sent1, [tokenizer.bos_token_id], sent2]).astype(np.int64) # [sent1, <bos> sent2]

  # label/gt to predict; -100 used for masking that input (in ground truth only)
  gt = np.concatenate([
      [MASK_TOKEN_ID for _ in sent1],
      [MASK_TOKEN_ID],
      [val if val != tokenizer.pad_token_id else MASK_TOKEN_ID for val in sent2]
  ]).astype(np.int64)

  # segment
  segment = np.concatenate([
      [tokenizer.additional_special_tokens_ids[0] for _ in sent1],
      [tokenizer.additional_special_tokens_ids[1]],
      [tokenizer.additional_special_tokens_ids[1] for _ in sent2]
  ]).astype(np.int64)

  exp["prefix_sent"] = sent1
  exp["suffix_sent"] = sent2

  exp["input"] = sentence_to_input_gpt2
  exp["label"] = gt
  exp["segment"] = segment

  return exp

In [ ]:
# read csv file
df = pd.read_csv('/content/drive/MyDrive/QnA/train_orig.csv')
df.head()

q = list(df["Question"])
a = list(df["Answer"])

mix_question_answer = [(q[i], a[i]) for i in range(len(q))]

In [ ]:
mix_question_answer_np = np.array(mix_question_answer)
np.random.shuffle(mix_question_answer_np)

In [ ]:
type(mix_question_answer_np)

numpy.ndarray

In [ ]:
mix_question_answer_np = mix_question_answer_np.tolist()

In [ ]:
# t = (1,2)
# print(t[0])

1


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split into train, validation and test
all_q = [mix_question_answer_np[i][0] for i in range(len(mix_question_answer_np))]
all_a = [mix_question_answer_np[i][1] for i in range(len(mix_question_answer_np))]


train_q, test_q, train_ans, test_ans = train_test_split(all_q, all_a, test_size=0.33, random_state=42)
train_q, val_q, train_ans, val_ans = train_test_split(train_q, train_ans, test_size=0.33, random_state=42)

In [ ]:
val_q, test_q = test_q, val_q
val_ans, test_ans = test_ans, val_ans

In [ ]:
print(len(train_q), len(test_q), len(val_q), sep="\n")
print(len(train_ans), len(test_ans), len(val_ans), sep="\n")

7364
3628
5415
7364
3628
5415


In [ ]:
# QNA dataset
class QnA_Dataset(Dataset):
    def __init__(self, qna_dataset_dir, config, tokenizer, ques, ans, limit_examples = None, evaluate = False, split_type = "train"):
      self.config = config
      self.examples = []

      split_data = [(ques[i], ans[i]) for i in range(len(ques))]
      self.examples = [input_to_dict(self.config, sample, tokenizer) for sample in tqdm.tqdm(split_data)]

      print("\n\n After conversion- ", self.examples[0])

      # Reduce dataset if required
      if limit_examples != None:
        self.examples = self.examples[:limit_examples]

      print("\n\n Doing Preprocess each sample")
      # do Preprocessing in each of the converted samples
      self.examples = [preprocess(exp, tokenizer, self.config, do_tokenize = True) for exp in self.examples]

      print("\n\n After preprocessing- ", self.examples[0])

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
      sentence = self.examples[idx]["input"]
      label = self.examples[idx]["label"]
      segment = self.examples[idx]["segment"]
      context_len = self.config["max_prefix_length"] + 1 # (+1) for <bos>

      return {
          "sample_number": idx,
          "sentence": torch.tensor(sentence),
          "label": torch.tensor(label),
          "segment": torch.tensor(segment)
      }

In [ ]:
# create DVP dataset
train_dataset = QnA_Dataset(qna_dataset_dir,
                                            INPUT_FORMAT_CONFIG,
                                            tokenizer,train_q, train_ans,
                                            limit_examples = None,
                                            evaluate = False, split_type = "train")
print("\n\n QnA Dataset created")

100%|██████████| 7364/7364 [00:31<00:00, 231.04it/s]




 After conversion-  {'sent1_tokens': [2061, 5640, 33636, 11880, 5633], 'sent2_tokens': [2061, 5640, 22987, 11880, 30, 1148, 340, 8513, 30, 383, 2748, 2728, 286, 22987, 11880, 468, 407, 587, 5174, 13, 2102, 11, 780, 262, 4006, 743, 307, 1944, 287, 1811, 1866, 286, 262, 976, 1641, 11, 25862, 743, 2620, 257, 1048, 338, 8395, 286, 5922, 262, 4006, 13, 317, 2050, 416, 1962, 320, 3565, 357, 12726, 8, 3751, 326, 257, 2176, 15304, 286, 257, 9779, 1444, 14639, 12, 16, 33, 357, 3849, 293, 2724, 259, 12, 16, 12159, 8, 318, 3917, 351, 281, 3220, 2526, 286, 5922, 22987, 11880, 290, 5644, 257, 8513, 4308, 329, 262, 2478, 286, 262, 4369, 13, 7735, 2267, 743, 1255, 287, 257, 1365, 4547, 286, 262, 8513, 16717, 2950, 287, 262, 2478, 286, 22987, 11880, 13]}


 Doing Preprocess each sample


 After preprocessing-  {'sent1_tokens': [2061, 5640, 33636, 11880, 5633], 'sent2_tokens': [2061, 5640, 22987, 11880, 30, 1148, 340, 8513, 30, 383, 2748, 2728, 286, 22987, 11880, 468, 407, 587, 5174, 13, 2102, 11, 78

In [ ]:
# create dataloader
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler = train_sampler, batch_size = train_batch_size)

print("QnA train dataloader created")

QnA train dataloader created


In [ ]:
# Total steps needed
t_total = len(train_dataloader) // gradient_accumulation_steps * num_epochs

# setting up the optimizer & learning rate schedulers
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']
grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters()],
        'weight_decay': 0.0
    }
]

optimizer = AdamW(grouped_parameters, lr = float(learning_rt), eps = adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_steps, num_training_steps = t_total)

print("Adam Optimizer and learning rate scheduler instantiated")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Adam Optimizer and learning rate scheduler instantiated


In [ ]:
# Training meta Information
print("Num of examples- ", len(train_dataset))
print("Num of epochs- ", num_epochs)
print("Batch size- ", train_batch_size)
print("Gradient acculmulation steps- ", gradient_accumulation_steps)
print("Total optimization steps- ", t_total)

Num of examples-  7364
Num of epochs-  3
Batch size-  5
Gradient acculmulation steps-  10
Total optimization steps-  441


In [ ]:
# zero out all the gradients
model.zero_grad()

In [ ]:
# Fn to save checkpoints
def save_model(model, tokenizer, chkpt_dir, global_step, args_dir):
  if not os.path.exists(chkpt_dir):
    os.makedirs(chkpt_dir)
  # print("Directory created for new checkpt to save")

  model.save_pretrained(chkpt_dir)
  tokenizer.save_pretrained(chkpt_dir)
  # print("Model and tokenizer saved")

  # save training arguments also
  with open(chkpt_dir + "/my_args.json", "w") as json_file:
    json.dump(args_dir, json_file)
  # print("Training arguments saved")

  with open(os.path.join(chkpt_dir, "global_step.txt"), "w") as f:
    f.write(str(global_step) + "\n")
  # print("Global step file saved")

  print("Checkpint saving process done..")

In [ ]:
from tqdm import tqdm

global_step = 0
train_loss_val = 0.0
chkpts_dir_name = []

# start training
train_iterator = trange(int(num_epochs), desc = "Epoch")
for epoch in train_iterator:
    epoch_iterator = tqdm(train_dataloader)

    for batch_idx, batch in enumerate(epoch_iterator):
      sentences = batch["sentence"].to(device)
      labels = batch["label"].to(device)
      segments = batch["segment"].to(device)
      model.train()

      outputs = model(input_ids=sentences, token_type_ids=segments, labels=labels)
      # print("Got logits and loss")

      loss = outputs.loss
      loss = loss / gradient_accumulation_steps
      train_loss_val += loss.item()

      loss.backward()

      if (((batch_idx + 1) % gradient_accumulation_steps) == 0):
        # print("Moved 1 step")
        F.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        model.zero_grad()
        global_step += 1

        if (global_step % save_steps == 0):
          # save checkpoint here
          print("Saving new checkpoint")
          chkpt_dir = qna_model_chkpts_dir + "/qna_chkpt_"+str(global_step)
          chkpts_dir_name.append("qna_chkpt_"+str(global_step))

          save_model(model, tokenizer, chkpt_dir, global_step, args_dir)
      # break

 14%|█▎        | 199/1473 [00:31<02:56,  7.24it/s]

Saving new checkpoint



 14%|█▎        | 200/1473 [00:33<16:41,  1.27it/s]

Checkpint saving process done..



 27%|██▋       | 399/1473 [01:02<02:29,  7.20it/s]

...


 99%|█████████▉| 1459/1473 [03:58<00:02,  6.91it/s]

Saving new checkpoint



 99%|█████████▉| 1460/1473 [04:02<00:15,  1.18s/it]

Checkpint saving process done..



Epoch: 100%|██████████| 3/3 [11:58<00:00, 239.39s/it]


In [ ]:
# Average train_loss per step
global_step, tr_loss = global_step, train_loss_val / (global_step) # +1 only while testing
print("Final Global step- ", global_step)
print("Average training loss per step- ", tr_loss)

Final Global step-  441
Average training loss per step-  2.2707876763359356


In [ ]:
# save the last model also
# global_step = 3  # remove this also; fetch the last global step value
chkpt_dir = qna_model_chkpts_dir + "/qna_chkpt_"+str(global_step)
chkpts_dir_name.append("qna_chkpt_"+str(global_step))

# save_model(model, tokenizer, chkpt_dir, args_dir)
save_model(model, tokenizer, chkpt_dir, global_step, args_dir)
print("Last model state saved")

Checkpint saving process done..
Last model state saved


In [ ]:
# Till now,
# QnA trained -> checkpoints saved -> Last model state saved
print("Checkpoints saved with the name- ", chkpts_dir_name)

Checkpoints saved with the name-  ['qna_chkpt_20', 'qna_chkpt_40', 'qna_chkpt_60', 'qna_chkpt_80', 'qna_chkpt_100', 'qna_chkpt_120', 'qna_chkpt_140', 'qna_chkpt_160', 'qna_chkpt_180', 'qna_chkpt_200', 'qna_chkpt_220', 'qna_chkpt_240', 'qna_chkpt_260', 'qna_chkpt_280', 'qna_chkpt_300', 'qna_chkpt_320', 'qna_chkpt_340', 'qna_chkpt_360', 'qna_chkpt_380', 'qna_chkpt_400', 'qna_chkpt_420', 'qna_chkpt_440', 'qna_chkpt_441']


In [ ]:
# Start Evaluation
print("Starting Evaluation of QnA on dev data based on perplexity")

Starting Evaluation of QnA on dev data based on perplexity


In [ ]:
# Get validation dataset and dataloader
import tqdm
val_dataset = QnA_Dataset(qna_dataset_dir,
                                            INPUT_FORMAT_CONFIG,
                                            tokenizer,val_q, val_ans,
                                            limit_examples = None,
                                            evaluate = True, split_type = "dev")

val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler = val_sampler, batch_size = eval_batch_size)

print("Validation dataset and dataloader created")

100%|██████████| 5415/5415 [00:15<00:00, 355.85it/s]




 After conversion-  {'sent1_tokens': [2061, 389, 262, 13820, 329, 2908, 268, 1287, 2011, 27189, 5633], 'sent2_tokens': [21327, 11, 691, 4318, 4755, 4369, 468, 281, 4050, 3513, 357, 3826, 2029, 737, 1318, 389, 645, 1900, 50019, 329, 597, 286, 777, 11916, 13, 7929, 425, 3513, 743, 6211, 29617, 19458, 291, 13820, 11, 355, 880, 355, 3518, 11, 34266, 393, 4046, 9102, 13]}


 Doing Preprocess each sample


 After preprocessing-  {'sent1_tokens': [2061, 389, 262, 13820, 329, 2908, 268, 1287, 2011, 27189, 5633], 'sent2_tokens': [21327, 11, 691, 4318, 4755, 4369, 468, 281, 4050, 3513, 357, 3826, 2029, 737, 1318, 389, 645, 1900, 50019, 329, 597, 286, 777, 11916, 13, 7929, 425, 3513, 743, 6211, 29617, 19458, 291, 13820, 11, 355, 880, 355, 3518, 11, 34266, 393, 4046, 9102, 13], 'prefix_sent': array([50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
       50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
       50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 502

In [ ]:
# Validation meta Information
print("Num of examples- ", len(val_dataset))
print("Batch size- ", eval_batch_size)

Num of examples-  5415
Batch size-  5


In [ ]:
# Fn to evaluate on a DVP checkpoint
def evaluate(model, tokenizer, chkpt_dir_name, val_dataloader):
  val_loss = 0.0
  model.eval()

  for i, batch in enumerate(val_dataloader):
    sentences = batch["sentence"].to(device)
    labels = batch["label"].to(device)
    segments = batch["segment"].to(device)

    with torch.no_grad():
      op = model(input_ids=sentences, token_type_ids=segments, labels=labels)
      loss_val = op.loss.item()

    val_loss += loss_val
    # break

  avg_val_loss = val_loss / (i + 1) # per batch average loss
  perplexity = torch.exp(torch.tensor(avg_val_loss)) # perplexity of exp(avg_loss)

  return perplexity

In [ ]:
chkpts_dir_name = chkpts_dir_name[2:]

In [ ]:
# Start evaluating the checkpoints on dev data and using perplexity as a measure
perplexity_list = []

for chkpt_name in chkpts_dir_name:

  print("Evaluating ", chkpt_name)
  chkpt_to_load = qna_model_chkpts_dir + chkpt_name
  model = model_class.from_pretrained(chkpt_to_load)
  tokenizer = tokenizer_class.from_pretrained(chkpt_to_load, do_lower_case = True)
  model.to(device)

  print("Checkpoint- " + chkpt_name + " loaded")

  # evaluate loaded
  print("Evaluating on loaded checkpoint")
  perplexity = evaluate(model, tokenizer, chkpt_dir, val_dataloader)
  perplexity_list.append((chkpt_name, perplexity))

print("QnA evaluated on all the saved checkpoints")

Evaluating  qna_chkpt_60


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Checkpoint- qna_chkpt_60 loaded
Evaluating on loaded checkpoint
Evaluating  qna_chkpt_80


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Checkpoint- qna_chkpt_80 loaded
Evaluating on loaded checkpoint
Evaluating  qna_chkpt_100


...

Checkpoint- qna_chkpt_420 loaded
Evaluating on loaded checkpoint
Evaluating  qna_chkpt_440


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Checkpoint- qna_chkpt_440 loaded
Evaluating on loaded checkpoint
Evaluating  qna_chkpt_441


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Checkpoint- qna_chkpt_441 loaded
Evaluating on loaded checkpoint
QnA evaluated on all the saved checkpoints


In [ ]:
# Sort perplexity list in increasing order to get best model
perplexity_list.sort(key=lambda x: x[1].item())
top_chkpt_name = perplexity_list[0][0]

print("Top performing checkpoint is- ", top_chkpt_name)

Top performing checkpoint is-  qna_chkpt_441


In [ ]:
# Evaluation on dev data done
chkpt_to_load = qna_model_chkpts_dir + top_chkpt_name
print(chkpt_to_load)

/content/drive/MyDrive/QnA/qna_chkpt_441


In [ ]:
# just to verify that copied model is loading correctly or not
chkpt_to_load = qna_model_chkpts_dir + top_chkpt_name
model = model_class.from_pretrained(chkpt_to_load)
tokenizer = tokenizer_class.from_pretrained(chkpt_to_load, do_lower_case = True)

print("Model loaded successfully..!!")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded successfully..!!


In [ ]:
# Get output logits from paraphraser
def get_logits(model, idx, sents, segments, past):
  if idx == 0:
      pred = model(input_ids = sents, token_type_ids = segments, return_dict=True)
  else:
      # used the cached representations to speed up decoding
      pred = model(input_ids = sents[:, -1:], token_type_ids = segments[:, -1:], past_key_values = past, return_dict = True)

  logits = pred['logits']
  past_keys = pred['past_key_values']

  return logits, past_keys

In [ ]:
# Decide generation lenght and get converted output and score
def generate(model, sents_to_paraphrase, segments, eos_token_id, top_p, top_k, len_to_gen):
  batch_size = sents_to_paraphrase.shape[0] # total sents in batch

  eos_emitted = [False for _ in range(batch_size)]
  scores = [{"score": 0, "sequence": []} for _ in range(batch_size)]

  with torch.no_grad():
    past_keys = None

    for i in range(len_to_gen):
      op_logits, past_keys = get_logits(model, i, sents_to_paraphrase, segments, past_keys)
      next_token_logits = op_logits[:, -1, :]
      original_scores = nn.Softmax(dim = -1)(next_token_logits)

      # do nucleas filtering and greedy decoding
      filtered_logits = next_token_logits
      next_token = torch.argmax(filtered_logits, dim = -1).unsqueeze(-1)

      # if top_k in [0, 1] and top_p == 0.0: # mainly to control the output diversity
      #   # greedy sampling
      #   next_token = torch.argmax(filtered_logits, dim = -1).unsqueeze(-1)
      # else :
      #   next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples = 1)

      sents_to_paraphrase = torch.cat((sents_to_paraphrase, next_token), dim=1)
      segments = torch.cat((segments, segments[:, -1:]), dim=1)

      for batch_elem in range(batch_size):
        if next_token[batch_elem].item() == eos_token_id:
            eos_emitted[batch_elem] = True

      if len_to_gen is None and all(eos_emitted):
        break

  return sents_to_paraphrase, scores

In [ ]:
# Preprocess input from to paraphrase
def preprocess2(exp, tokenizer, config):
  max_prefix_len = config["max_prefix_length"]
  sent1 = np.array(exp["sent1_tokens"])

  # truncate
  if(len(sent1) > max_prefix_len):
    sent1 = sent1[:max_prefix_len]

  # add padding; left padding to prefix and right padding to suffix
  count_pad_tokens_prefix = max_prefix_len - len(sent1)
  sent1 = np.pad(sent1, (count_pad_tokens_prefix, 0), constant_values = tokenizer.pad_token_id)

  # sentence to input gpt2
  sentence_to_input_gpt2 = np.concatenate([sent1, [tokenizer.bos_token_id]]).astype(np.int64) # [sent1, <bos>]

  # segment
  segment = np.concatenate([
      [tokenizer.additional_special_tokens_ids[0] for _ in sent1],
      [tokenizer.additional_special_tokens_ids[1]]
  ]).astype(np.int64)

  exp["input"] = sentence_to_input_gpt2
  exp["segment"] = segment

  return exp

In [ ]:
# Generate paraphrased sentences for batch of input sentences
def generate_paraphrased_sent(model, top_p, top_k, sents, config, device, tokenizer):
  examples = []

  for sent in sents:
    token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))
    dd = {"sent1_tokens":token_ids}

    dd_process = preprocess2(dd, tokenizer, config)
    examples.append(dd_process)

  init_context_size = 1 + config["max_prefix_length"]

  output, scores = generate(model.to(device),
      sents_to_paraphrase = torch.tensor([inst["input"] for inst in examples]).to(device),
      segments = torch.tensor([inst["segment"] for inst in examples]).to(device),
      eos_token_id = tokenizer.eos_token_id,
      top_p = top_p, top_k = top_k,
      len_to_gen =  config["max_suffix_length"] + 1  # +1 for <eos>
  )

  all_ops = []
  for idx in range(len(output)):
    exmp = examples[idx]
    curr_out = output[idx, init_context_size:].tolist()

    if tokenizer.eos_token_id in curr_out:
      curr_out = curr_out[:curr_out.index(tokenizer.eos_token_id)]

    all_ops.append(tokenizer.decode(curr_out, clean_up_tokenization_spaces = True, skip_special_tokens = True))

  return all_ops, scores

In [ ]:
top_p = 0.0
top_k = 1.0

In [ ]:
x,y = generate_paraphrased_sent(model, top_p, top_k, ["How to diagnose Parasites - Cysticercosis?"], INPUT_FORMAT_CONFIG, device, tokenizer)

In [ ]:
print(x)

["How is cysticercosis diagnosed? Cysticercosis is diagnosed by a doctor's examination of the skin and mucous membranes. The doctor will examine the skin and mucous membranes to find out if the cysticerc"]
